In [3]:
# Import packages
from torch import nn 
# from torch.nn import functional as F
from torch import optim 
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from data import train_data, test_data
from residual_block import Residual_block
from resnet18 import ResNet18

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '/data'

In [ ]:
# Create DATALoader
train_loader = DataLoader(train_data, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 16, shuffle = False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

train_losses = []
test_losses = []

#define ResNet18
net = ResNet().to(device)

#set hyperparameter
EPOCH = 2
LR = 0.001

#define loss funtion & optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR)                      # momentum=0.9, weight_decay=5e-4


In [ ]:
for epoch in range(EPOCH):
   
    print('\nEpoch_train: %d' % (epoch + 1))
   
    net.train()
    train_loss = 0.0
    test_loss = 0.0
    max_train_acc = 0
    max_test_acc = 0
    correct = 0.0
    correct1 = 0.0
    train_samples = 0
    
    for i, data in enumerate(train_loader, 0):
        if i == 5:
            break
        #prepare dataset
        length = len(train_loader)
        inputs, labels = data 
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        #forward & backward
        outputs = net(inputs)
        loss = loss_func(outputs, labels)          
        loss.backward()
        optimizer.step()
        
        #print ac & loss in each batch
        train_loss += loss.item() * outputs.size(0)                  # train_loss+= mean_batch_loss * batch_size 
                                                                   # Умножение на outputs.size(0) (батч) это по сути преобразование среднего значения ЛОСС за батч в
                                                                   #  суммарное, поскольку для подсчета Лосс за прогон суммарное за батч делиться на общее количество
                                                                   # фоток в прогоне, а не на количество итераций.
        train_samples += outputs.size(0)                           #подсчитвыет количество фото
        _, predicted = torch.max(outputs.data, 1)                  # _, predicted - значение тензора, номер индекса с этим максимальным значением
                                                                    # .data связывает между собой два тензора
        correct += predicted.eq(labels.data).cpu().sum()          # суммирует количество совпадающих 1 с labels
        acc_train = 100*correct / train_samples                    # точность 100. * correct / train_samples  
              
                      
        
        if acc_train > max_train_acc:
            max_train_acc = acc_train
            min_train_loss = train_loss / train_samples
                              
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.2f%% ' 
              % (epoch + 1, (i + 1 ), train_loss / train_samples, acc_train))                
        
    train_losses.append(train_loss / train_samples)
    

    print('\nEpoch_test: %d' % (epoch + 1))    
    
    with torch.no_grad():
        correct = 0
        test_samples = 0
        for i, data in enumerate(test_loader):          
            if i == 5:
                break
            net.eval()
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            test_loss += loss.item() * outputs.size(0)
            test_samples += outputs.size(0)
            correct += (predicted == labels).sum()
            acc_test = 100*correct / train_samples    
            
            print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.2f%% ' 
              % (epoch + 1, (i + 1 + epoch * length), test_loss / test_samples,  acc_test))        
    
            if acc_test > max_test_acc:
                max_test_acc = acc_test
                min_test_loss = test_loss / test_samples
                torch.save(net, f'model_my_resnet18.pth')            # сохраняет модель с лучшими показателями
    
        
    test_losses.append(test_loss / test_samples)
        
    print('------------')
    print('MAX_train_acc: %.2f%% ' % (max_train_acc))      
    print('MAX_test_acc: %.2f%% ' % (max_test_acc))
    print('MIN_train_loss: %.3f ' % (min_train_loss))      
    print('MIN_test_loss: %.3f ' % (min_test_loss))   
  
        
        

In [ ]:
plt.plot(train_losses, label="Training Loss",linewidth=2)
plt.plot(test_losses, label="Test Loss",linewidth=2)
plt.title('Training and Test Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()